In [1]:
import pandas as pd
import numpy as np
import os
from torchmetrics.functional import accuracy
from torchmetrics import Recall, F1Score, Precision
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
import numpy as np
from transformers import BertTokenizer

In [2]:
model = torch.load('model')

In [3]:
fake_text_all = pd.read_csv('DataSet_Misinfo_FAKE.csv').loc[1001:1101,:]
real_text_all = pd.read_csv('DataSet_Misinfo_TRUE.csv').loc[1001:1101,:]

fake_text_all.drop(fake_text_all.columns[0],axis=1,inplace=True)
real_text_all.drop(real_text_all.columns[0],axis=1,inplace=True)

In [4]:
mixed_text_all = pd.concat([real_text_all,fake_text_all])['text'].tolist()

for i in range(0,len(mixed_text_all)):
    mixed_text_all[i] = str(mixed_text_all[i])
    
Y_all = np.concatenate([np.ones(len(real_text_all)), np.zeros(len(fake_text_all))])

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')#选用large-cased版本
tokenized_text_all = tokenizer(mixed_text_all, padding='max_length', truncation=True, max_length=512, return_tensors='pt')


#构建TensorDataset用于训练
input_ids_all = tokenized_text_all['input_ids']
attention_mask_all = tokenized_text_all['attention_mask']
Y_all = torch.tensor(Y_all)

data_all = torch.utils.data.TensorDataset(input_ids_all,attention_mask_all,Y_all)

In [6]:
valdata_all = DataLoader(data_all, batch_size=32)

In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

model = torch.load("model")

for allinput_ids, allattention_mask, allY in tqdm(valdata_all):
    allinput_ids = allinput_ids.squeeze(1).to(device)
    allattention_mask = allattention_mask.to(device)
    allY = allY.to(device)
        
    optimizer.zero_grad()
    output_all = model(allinput_ids, allattention_mask)
            
    logits_all = output_all.logits
    predictions_all = torch.argmax(logits_all, dim=1)
    acc_all = accuracy(predictions_all, allY,'binary', num_classes=2)
                
    precision =  Precision('multiclass',average='micro', num_classes=2)
    pre_all = precision(predictions_all, allY)
    f1score =  F1Score('multiclass', num_classes=2)
    f1_all = f1score(predictions_all, allY)
    recall = Recall('multiclass',average='micro', num_classes=2)
    rec_all = recall(predictions_all, allY)

print(f'''| All-text Accuracy: {acc_all: .3f}  
          | All-text Precision: {pre_all: .3f} 
          | All-text Recall: {rec_all: .3f} 
          | All-text F1Score: {f1_all: .3f}''') 

 14%|█▍        | 1/7 [00:18<01:53, 18.93s/it]